# Taller 1

## Integrantes:
- Catalina Sanchez
- Samuel Fuentes
- Pamela Vera

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [2]:
# Datos a extraer de los Tequilas
data = {
    "Name": [],
    "Type": [],
    "Cask": [],
    "Location": [],
    "Age": [],
    "ABV %": [],
    "Cost": [],
    "Badge": [],
    "# Ratings": [],
    "Community's Rating": [],
    "Flavor Summary": [],
    "Expert": [],
    "Expert Score": [],
    "Smoky": [],
    "Earthy": [],
    "Spicy": [],
    "Herbal": [],
    "Oily": [],
    "Bitter": [],
    "Rich": [],
    "Sweet": [],
    "Mineral": [],
    "Salty": [],
    "Umami": [],
    "Tart": [],
    "Fruity": [],
    "Floral": [],
    "Review": []
}

In [3]:

# Inicializa una lista para almacenar los datos de todos los tequilas.
all_tequilas = []

# Configura el controlador de Selenium para usar Google Chrome.
driver = webdriver.Chrome()

# Maximiza la ventana del navegador para asegurar que todo el contenido sea visible.
driver.maximize_window()

# Abre la página web de búsqueda en Distiller.
driver.get("https://distiller.com/search")

# Espera 2 segundos para que la página cargue completamente.
time.sleep(2)


# Encuentra y cierra el botón de anuncios emergentes.
#driver.find_element(By.CLASS_NAME, 'adthrive-close').click()

# Encuentra y cierra el botón "Remind Me Later" para notificaciones.
driver.find_element(By.ID, 'onesignal-slidedown-cancel-button').click()

# Espera 1 segundo antes de proceder.
time.sleep(1)

# Encuentra y acepta el botón de privacidad.
driver.find_element(By.ID, 'accept-privacy').click()

# Espera 1 segundo más para que los cambios surtan efecto.
time.sleep(1)

# Selecciona la categoría "Tequila/Mezcal" en el menú de la página.
driver.find_element(By.XPATH, "//button[@data-value='agave']").click()

# Función para extraer los datos de un tequila en particular.
def get_tequila_data():
    try:
        # Inicializa el diccionario data y extrae los datos en un solo paso
        data = {
            "Name": driver.find_element(By.CSS_SELECTOR, "h1.secondary-headline.name").text,
            "Type": (driver.find_elements(By.CSS_SELECTOR, "p.ultra-mini-headline.type")[0].text 
                     if driver.find_elements(By.CSS_SELECTOR, "p.ultra-mini-headline.type") else None),
            "Location": (driver.find_elements(By.CSS_SELECTOR, "p.ultra-mini-headline.location")[0].text 
                        if driver.find_elements(By.CSS_SELECTOR, "p.ultra-mini-headline.location") else None),
            "Community's Rating": (driver.find_elements(By.CSS_SELECTOR, "div.rating-display__value span[itemprop='ratingValue']")[0].text 
                                if driver.find_elements(By.CSS_SELECTOR, "div.rating-display__value span[itemprop='ratingValue']") else None),
            "# Ratings": (driver.find_elements(By.CSS_SELECTOR, "span[itemprop='ratingCount']")[0].text 
                        if driver.find_elements(By.CSS_SELECTOR, "span[itemprop='ratingCount']") else None),
            "Expert": (driver.find_elements(By.CSS_SELECTOR, "div.meet-experts a[itemprop='author']")[0].text 
                    if driver.find_elements(By.CSS_SELECTOR, "div.meet-experts a[itemprop='author']") else None),
            "Expert Score": (driver.find_elements(By.CSS_SELECTOR, "div.distiller-score span")[0].text 
                            if driver.find_elements(By.CSS_SELECTOR, "div.distiller-score span") else None),
            "ABV %": (driver.find_elements(By.CSS_SELECTOR, "li.detail.abv div.value")[0].text 
                    if driver.find_elements(By.CSS_SELECTOR, "li.detail.abv div.value") else None),
            "Cost": None,
            "Cask": (driver.find_elements(By.CSS_SELECTOR, "li.detail.cask-type div.value")[0].text 
                    if driver.find_elements(By.CSS_SELECTOR, "li.detail.cask-type div.value") else None),  
        }

        cost_element = driver.find_elements(By.CLASS_NAME, "spirit-cost")
        if cost_element:
            cost_class = cost_element[0].get_attribute("class")
            data["Cost"] = cost_class.split("cost-")[-1]  # Obtiene el valor después de "cost-"

        
        return data
    except Exception as e:
        print(f"Error al extraer datos: {e}")
        return None
        
    except Exception as e:
        # Maneja cualquier error durante la extracción de datos y lo imprime en la consola.
        print(f"Error al obtener datos del tequila: {e}")
        return None  # Retorna None si ocurre un error.

# Bucle principal para recorrer todas las páginas de resultados.
for _ in range(1):  
    print("Scraping...")  # Muestra en consola que el scraping ha comenzado.
    try:
        # Encuentra todos los tequilas visibles en la página actual.
        spirits = driver.find_elements(By.CLASS_NAME, "spirit")
        print(f"Tequilas encontrados: {len(spirits)}")  # Muestra la cantidad de tequilas encontrados.

        # Itera sobre cada tequila en la lista.
        for i in range(len(spirits)):
            spirits = driver.find_elements(By.CLASS_NAME, "spirit")  # Actualiza la lista de tequilas (por si el DOM cambió).
            link = spirits[i].find_element(By.TAG_NAME, 'a')  # Encuentra el enlace al perfil del tequila.
            link.click()  # Haz clic en el enlace para acceder a los detalles del tequila.
            time.sleep(2)  # Espera 2 segundos para que la página cargue.

            # Extrae los datos del tequila y los agrega a la lista principal.
            tequila_data = get_tequila_data()
            if tequila_data:
                all_tequilas.append(tequila_data)
                print(f"Tequila {i+1} de {len(spirits)} extraído con éxito.")
                print(tequila_data)

            # Regresa a la página anterior después de extraer los datos.
            driver.back()
            time.sleep(2)

        # Encuentra y haz clic en el botón "Next" para ir a la siguiente página.
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"]')
        next_button.click()
        time.sleep(3)  # Espera 3

    except Exception as e:
        # Termina el bucle si ocurre un error (por ejemplo, no hay más páginas).
        print(f"Finalizando scraping: {e}")
        break

# Convierte la lista de diccionarios en un DataFrame de Pandas.
data_scraped = pd.DataFrame(all_tequilas)

# Guarda los datos en un archivo CSV para análisis posterior.
data_scraped.to_csv('data_scraped.csv', index=False)

# Cierra el navegador para finalizar el proceso.
driver.quit()


Scraping...
Tequilas encontrados: 10
Tequila 1 de 10 extraído con éxito.
{'Name': '4 COPAS TEQUILA BLANCO', 'Type': 'TEQUILA BLANCO', 'Location': '4 COPAS // JALISCO, MEXICO', "Community's Rating": '4.37', '# Ratings': '44', 'Expert': 'JACK ROBERTIELLO', 'Expert Score': '98', 'ABV %': '40.0', 'Cost': '2', 'Cask': 'UNAGED'}
Tequila 2 de 10 extraído con éxito.
{'Name': 'MEZCAL LOS SIETE MISTERIOS PECHUGA', 'Type': 'MEZCAL JOVEN', 'Location': 'LOS SIETE MISTERIOS // OAXACA, MEXICO', "Community's Rating": '4.41', '# Ratings': '37', 'Expert': 'OMARI WHEAT', 'Expert Score': '98', 'ABV %': '58.5', 'Cost': '5', 'Cask': 'NONE'}
Tequila 3 de 10 extraído con éxito.
{'Name': 'DEL MAGUEY ARROQUEÑO MEZCAL', 'Type': 'MEZCAL JOVEN', 'Location': 'DEL MAGUEY // OAXACA, MEXICO', "Community's Rating": '4.5', '# Ratings': '71', 'Expert': 'OMARI WHEAT', 'Expert Score': '98', 'ABV %': '49.0', 'Cost': '4', 'Cask': ''}
Tequila 4 de 10 extraído con éxito.
{'Name': 'FUENTESECA RESERVA EXTRA AÑEJO TEQUILA 9 AÑOS'